# More Raw Results from Vector

* Open an Terminal:

```sh
conda activate DEIO
pip install ipykernel 
```

* Choose the DEIO as Kernel for this `.ipynb` file

In [1]:
import os
import tqdm as tqdm
import numpy as np
import copy

# 处理服务器中evo的可视化问题
import evo
from evo.tools.settings import SETTINGS
SETTINGS['plot_backend'] = 'Agg'

from evo.tools import file_interface, plot
from evo.core.geometry import GeometryException
import evo.main_ape as main_ape
from evo.core import sync, metrics
from evo.core.trajectory import PoseTrajectory3D

import matplotlib.pyplot as plt #绘图

print("Successfully import ultils")

Successfully import ultils


In [2]:
def make_evo_traj_gt(poses_N_x_7, tss_us):
    assert poses_N_x_7.shape[1] == 7
    assert poses_N_x_7.shape[0] > 10
    assert tss_us.shape[0] == poses_N_x_7.shape[0]

    traj_evo = PoseTrajectory3D(
        positions_xyz=poses_N_x_7[:,:3],
        # orientations_quat_wxyz=poses_N_x_7[:,3:],
        orientations_quat_wxyz = poses_N_x_7[:, [6,3,4,5]],#gt存储的是xyzw
        timestamps=tss_us/1e6)#转换为秒
    return traj_evo

def make_evo_traj_deio(poses_N_x_7, tss_us):
    assert poses_N_x_7.shape[1] == 7
    assert poses_N_x_7.shape[0] > 10
    assert tss_us.shape[0] == poses_N_x_7.shape[0]

    traj_evo = PoseTrajectory3D(
        positions_xyz=poses_N_x_7[:,:3],
        # orientations_quat_wxyz=poses_N_x_7[:,3:],
        orientations_quat_wxyz = poses_N_x_7[:, [5,6,3,4]],#存储的是yzwx(由于原代码的bug导致的,注意统一代码输出为xyzw而evo中需要用的是wxyz即可)
        timestamps=tss_us/1e6)#转换为秒
    return traj_evo

def load_gt_us(path, skiprows=0):
    traj_ref = np.loadtxt(path, delimiter=" ", skiprows=skiprows)
    tss_gt_us = traj_ref[:, 0].copy() 
    assert np.all(tss_gt_us == sorted(tss_gt_us))
    assert traj_ref.shape[0] > 0
    assert traj_ref.shape[1] == 8

    return tss_gt_us, traj_ref[:, 1:]

def plot_trajectory_inxyplane(pred_traj, gt_traj, align=True, _n_to_align=-1,correct_scale=True, max_diff_sec=1.0,title="", filename=""):
    #将两个轨迹对齐(时间维度上的)
    gt_traj, pred_traj = sync.associate_trajectories(gt_traj, pred_traj, max_diff=max_diff_sec)

    # 对齐轨迹(空间维度上的)
    if align:
        try:
            pred_traj.align(gt_traj, correct_scale=correct_scale,n=_n_to_align)
        except GeometryException as e:
            print("Plotting error:", e)

    plot_collection = plot.PlotCollection("PlotCol")

    fig = plt.figure(figsize=(8, 8))
    plot_mode = plot.PlotMode.xy
    ax = plot.prepare_axis(fig, plot_mode)
    ax.set_title(title)
    if gt_traj is not None:
        plot.traj(ax, plot_mode, gt_traj, '--', 'gray', "Ground Truth")
    plot.traj(ax, plot_mode, pred_traj, '-', 'blue', "Predicted")
    
    plot_collection.add_figure("traj (error)", fig)

    plt.show()

print("Successfully define several functions")

Successfully define several functions


In [29]:
print("Evaluation for VECtor dataset")
data_dir="/media/lfl-data2/VECtor/" # your dataset path
cur_dir=os.getcwd() #current file directoin

print("current:",cur_dir)

target_dirs = {
                "corner_slow",
                "desk_normal",
                "sofa_fast",
                "mountain_fast",
                "corridors_dolly",
                "corridors_walk",
                "units_dolly",
                "units_scooter",
                "robot_normal",
                "robot_fast",
                "desk_fast",
                "sofa_normal",
                "mountain_normal",
                "hdr_normal",
                "hdr_fast",
                }

for root, dirs, files in os.walk(data_dir):
    for d in dirs:
        # 构建完整路径 data_path
        datapath_val = os.path.join(root, d)

        # 检查是否为目标文件夹之一
        if os.path.basename(datapath_val) in target_dirs:
            sequence_name = os.path.basename(datapath_val)

            # 获取轨迹
            tss_gt_us, traj_gt = load_gt_us(os.path.join(datapath_val, f"poses_evs_left.txt"))#获取真实轨迹

            # 获取deio估算的轨迹
            if sequence_name == "corner_slow":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "VECtor/2024-10-13_corner_slow1/Corner_Slow1_Trial01.txt"))
            elif sequence_name == "desk_normal":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-13_desk_normal1/Desk_Normal1_Trial01.txt"))
            elif sequence_name == "sofa_fast":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-10_sofa_fast1/Sofa_Fast1_Trial01.txt"))
            elif sequence_name == "mountain_fast":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-14_mountain_fast1/Mountain_Fast1_Trial01.txt"))
            elif sequence_name == "corridors_dolly":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-12_corridors_dolly1/Corridors_Dolly1_Trial01.txt"))
            elif sequence_name == "corridors_walk":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-12_corridors_walk1/Corridors_Walk1_Trial01.txt"))
            elif sequence_name == "units_dolly":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-12_units_dolly1/Units_Dolly1_Trial01.txt"))
            elif sequence_name == "units_scooter":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/2024-10-12_units_scooter1/Units_Scooter1_Trial01.txt"))
            elif sequence_name == "robot_normal":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Robot_Normal1_trial_0_step_DEVO/Robot_Normal1_Trial00.txt"))
            elif sequence_name == "robot_fast":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Robot_Fast1_trial_3_step_DEVO/Robot_Fast1_Trial03.txt"))
            elif sequence_name == "desk_fast":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Desk_Fast1_trial_3_step_DEVO/Desk_Fast1_Trial03.txt"))
            elif sequence_name == "sofa_normal":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Sofa_Normal1_trial_1_step_DEVO/Sofa_Normal1_Trial01.txt"))   
            elif sequence_name == "mountain_normal":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Mountain_Normal1_trial_1_step_DEVO/Mountain_Normal1_Trial01.txt"))
            elif sequence_name == "hdr_normal":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Hdr_Normal1_trial_4_step_DEVO/Hdr_Normal1_Trial04.txt"))  
            elif sequence_name == "hdr_fast":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./VECtor/Hdr_Fast1_trial_4_step_DEVO/Hdr_Fast1_Trial04.txt")) 

            evoGT = make_evo_traj_gt(traj_gt, tss_gt_us)
            evoEst = make_evo_traj_deio(traj_deio, tss_deio_us)
            gtlentraj = evoGT.get_infos()["path length (m)"]#获取轨迹长度
            evoGT, evoEst = sync.associate_trajectories(evoGT, evoEst, max_diff=1)
            _n_to_align=-1;
            ape_trans = main_ape.ape(copy.deepcopy(evoGT), copy.deepcopy(evoEst), pose_relation=metrics.PoseRelation.translation_part, align=True,n_to_align=_n_to_align, correct_scale=True)

            # print(f"\033[31m EVO结果：{ape_trans}\033[0m");
            MPE = ape_trans.stats["mean"] / gtlentraj * 100
            # print(f"MPE is {MPE:.02f}") #注意只保留两位小数
            evoATE = ape_trans.stats["rmse"]*100

            res_str = f"\nATE[cm]: {evoATE:.02f} | MPE[%/m]: {MPE:.02f}"
            
            print(f"{sequence_name}: {res_str}")
            
    # 使用break限制os.walk只遍历indir的第一层
    break



Evaluation for VECtor dataset
current: /home/gwp/DEIO/estimated_trajectories


corner_slow: 
ATE[cm]: 0.87 | MPE[%/m]: 0.50
corridors_dolly: 
ATE[cm]: 74.88 | MPE[%/m]: 0.78
corridors_walk: 
ATE[cm]: 68.50 | MPE[%/m]: 0.74
desk_fast: 
ATE[cm]: 4.09 | MPE[%/m]: 0.10
desk_normal: 
ATE[cm]: 1.39 | MPE[%/m]: 0.13
hdr_fast: 
ATE[cm]: 5.76 | MPE[%/m]: 0.22
hdr_normal: 
ATE[cm]: 1.43 | MPE[%/m]: 0.33
mountain_fast: 
ATE[cm]: 8.87 | MPE[%/m]: 0.24
mountain_normal: 
ATE[cm]: 11.06 | MPE[%/m]: 0.69
robot_fast: 
ATE[cm]: 2.36 | MPE[%/m]: 0.09
robot_normal: 
ATE[cm]: 0.82 | MPE[%/m]: 0.16
sofa_fast: 
ATE[cm]: 16.52 | MPE[%/m]: 0.44
sofa_normal: 
ATE[cm]: 4.82 | MPE[%/m]: 0.12
units_dolly: 
ATE[cm]: 101.19 | MPE[%/m]: 0.35
units_scooter: 
ATE[cm]: 101.16 | MPE[%/m]: 0.35
